# Cross-Validation 

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Regularization').getOrCreate()

In [ ]:
sc.install_pypi_package("sklearn")

In [ ]:
sc.install_pypi_package("pandas==0.25.1")

In [ ]:
sc.install_pypi_package("matplotlib==3.2.2")

In [ ]:
from sklearn.datasets import load_diabetes
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['y'] = diabetes.target
sdf = spark.createDataFrame(df)

assembler = VectorAssembler(inputCols=[c for c in sdf.columns if c != 'y'],
                            outputCol='features')
dataset = assembler.transform(sdf)
dataset.show(5)

## We haven't tested models on unseen data!

In [ ]:
# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)


In [ ]:
train, test = dataset.randomSplit([0.5, 0.5], 25)
lr = LinearRegression(featuresCol='features', labelCol='y')
model = lr.fit(train)

# Evaluate on training data
summary_train = model.evaluate(train)
print('R2 (training): ', summary_train.r2)

# Evaluate on training data
summary_test = model.evaluate(test)
print('R2 (testing): ', summary_test.r2)

## Increase training fraction?

In [ ]:
# We specify this so that the train and test data set always have the same rows, respectively
# np.random.seed(0)

In [ ]:
plt.clf()

In [ ]:
p_train = [0.6, 0.7, 0.8, 0.9]
train_r2 = []
test_r2 = []

for p in p_train:
    train, test = dataset.randomSplit([p, 1 - p], 25)
    model = lr.fit(train)
    
    # Evaluate on training data
    summary_train = model.evaluate(train)
    train_r2.append(summary_train.r2)

    # Evaluate on training data
    summary_test = model.evaluate(test)
    test_r2.append(summary_test.r2)

plt.plot(p_train, train_r2, 'ro-', label='train')
plt.plot(p_train, test_r2, 'g', label='test')
plt.xlabel('Training Size')
plt.ylabel('R2')
plt.legend()

In [ ]:
%matplot plt